# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
#прописываем нужные нам библиотеки 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Разбейте данные на выборки

In [4]:
features = df.drop(['is_ultra'], axis =1)
target = df['is_ultra']


features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, random_state=12345)
features_test, features_valid, target_test, target_valid = train_test_split(features_test, target_test, test_size=0.5, random_state=12345)

print('Обучающая выборка:',features_train.shape)
print(target_train.shape)
print('Валидационная выборка:',features_valid.shape)
print(target_valid.shape)
print('Тестовая выборка:',features_test.shape)
print(target_test.shape)

Обучающая выборка: (1928, 4)
(1928,)
Валидационная выборка: (643, 4)
(643,)
Тестовая выборка: (643, 4)
(643,)


Разбили нашу выборку в соотношении 3:1:1

## Исследуйте модели

In [5]:
#Дерево решений
for depth in range(1, 6):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
   
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid)) 

max_depth = 1 : 0.7356143079315708
max_depth = 2 : 0.7744945567651633
max_depth = 3 : 0.7791601866251944
max_depth = 4 : 0.7744945567651633
max_depth = 5 : 0.7838258164852255


Наилочная точность выборки оказалась 0.78, при глубине дерева 5

In [6]:
# Инициализируем  модель случайного леса
best_model = None
best_est = 0
best_result = 0
for est in range(10, 60):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train,target_train) 
    result = model.score(features_valid,target_valid) 
    if result > best_result:
        best_model = model 
        best_result = result 
        best_est = est

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "При количестве деревьев:", best_est)

Accuracy наилучшей модели на валидационной выборке: 0.7962674961119751 При количестве деревьев: 48


In [7]:
model = LogisticRegression(random_state=12345)# инициализируйте модель логистической регрессии с параметром random_state=12345
model.fit(features_train,target_train) # обучите модель на тренировочной выборке
result = model.score(features_valid,target_valid) # получите метрику качества модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.6842923794712286


**Пока что можно сделать вывод, что лучше всего себя показывает метод случайного леса**

## Проверьте модель на тестовой выборке

In [8]:
#Дерево решений
model_dt=DecisionTreeClassifier(random_state=12345, max_depth=5, min_samples_leaf=1, min_samples_split= 3)
model_dt.fit(features_train, target_train)
accuracy_dt=model_dt.score(features_test, target_test)
print(accuracy_dt)

0.7791601866251944


In [9]:
#Случайный лес
model_dt=RandomForestClassifier(random_state=12345, max_depth=10, min_samples_leaf=1, min_samples_split= 3)
model_dt.fit(features_train, target_train)
accuracy_dt=model_dt.score(features_test, target_test)
print(accuracy_dt)

0.7993779160186625


In [10]:
#логическая регрессия
model_dt=LogisticRegression(random_state=12345)
model_dt.fit(features_train, target_train)
accuracy_dt=model_dt.score(features_test, target_test)
print(accuracy_dt)

0.7107309486780715


**Лучше всего себя показал метод "Случайный лес" с точность практически 80%, хоть ему и потребовалось дольше времени на создание модели и расчеты.**

**Следом, с небольшим отстованием, метод "Дерево решений" с точность 78%, этот метод подойдет при необходимости получить быстрый результат**

## (бонус) Проверьте модели на адекватность

In [11]:

dummy_clf = DummyClassifier(strategy="most_frequent", random_state=0)
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_train, target_train)

0.6924273858921162

**Используя фиктивный классификатор, со стратегией предсказывания самого частого класса в данных обучения, мы получили эффективность предсказания 69%, что указывает на адекватность всех трех обученных моделей**

**Модель логистической регрессии имеет точность предсказания всего на пару процентов выше чем квазиклассификатор.**

## Вывод:

1. Наилучшую эффективность показала модель по методу "Случайный лес", с точностью предсказания практически 80%.

2. Для получения быстрых результатов, предлагаю использовать модель по методу "Дерево решений", с точностью предсказания практически 78%, т.к. моделе "Случайный лес" требуется больше вычислительных ресурсов и времени